# Single-key models

In [35]:
import os

import keras
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
import tensorflow as tf
from tensorflow.keras.losses import MSLE

print("TensorFlow version:", tf.version.VERSION)

TensorFlow version: 2.16.2


In [2]:
plotly.offline.init_notebook_mode(connected=True)  # for nbviewer
plotly.io.templates.default = "plotly_dark"

_ = pl.Config(set_float_precision=2)

# Introduction
---

In this notebook, we build models to predict the target values of a single key at a time.

The models we build will successively (or in batch) take the inputs for a single key and produce predictions for the `sales` values for this key.

We use only a limited number of covariates as input.
Most notably, we do not include information about the key in the input data,
that is the values of `store_nbr` and `family` are not known to the model when training or making predictions.
This has the advantage of greatly reducing the number of input features (and thus the width of our model), 
but comes at a cost: our models will be incapable of discerning the key-specific (or store/family-specific)
properties of the series.
This is particularly noticeable when considering families with different significant time frequencies.
Indeed, our models will not be able to distinguish between families with strong weekly periodicity and families with no
periodicity at all, which will result in very "average" predictions, i.e. predictions which do not fit any key
particularly well but which attain a reasonable loss for the vast majority of keys.

In the next notebook, we will construct similar models (predicting one key at a time), but include 
the key information in the input (by one-hot encoding the `store_nbr` and `family` columns).


We choose to use 64 days of input, to predict the 16 days of the target.

In [3]:
TARGET_STEPS = 16  # length of the target sequences
INPUT_STEPS = 64  # length of input sequences (chosen)

Read the dataframe created in the first notebook and select the features used here.

We use a `weekday` column instead of the weekly time signals included in the dataframe because
the models in this notebook come from a previous iteration of this project and we need the same
features to use the trained models.
The previous iteration can be found in the extras folder and describes in more details the process
of selecting and training the models.

In [4]:
# read the dataframe created in the previous notebook
df = pl.read_csv(os.path.join("input", "df.csv"), try_parse_dates=True)

df = df.select("date", "sales", "onpromotion", "dcoilwtico", "store_nbr", "family")

# add a "weekday" column containing the day of the week
df = df.with_columns(pl.col("date").dt.weekday().alias("weekday"))

In [5]:
df.schema

Schema([('date', Date),
        ('sales', Float64),
        ('onpromotion', Int64),
        ('dcoilwtico', Float64),
        ('store_nbr', Int64),
        ('family', String),
        ('weekday', Int8)])

# Data preparation
---

We store our data in a custom `DataTensor` class.

The data is stored in a tensor with axes (time, key, feature), and is split into training/validation/test/target sets.

Note that we do not normalize the data, although it would seem to make sense, as we achieve better results with
unnormalized data (confirmed through experimentation).
This is worth noting as one might expect the opposite to be true, especially since using unnormalized data
means that our models will see a wide range of values for the `sales` column, which varies greatly from key to key.
However, doing things this way makes our predictions directly scale with the corresponding input, 
and therefore avoid having small errors grow large when unscaling keys with large sales values.

The `split` parameter of the `DataTensor` class shoud contain the proportion of training data to use
for the training and validation sets respectively.
The remainder of the training data is used as test set.

In [6]:
class DataTensor:
    def __init__(self, df: pl.DataFrame, split: tuple[float, float] = (0.7, 0.2)):
        dfs = df.partition_by(
            ["store_nbr", "family"], maintain_order=True, include_key=False
        )

        # stack into a tensor with axes (time, key, feature)
        self.data = tf.stack([tf.constant(df, dtype=tf.float32) for df in dfs], axis=1)

        # compute number of time-steps in each of train/valid/test/target sets
        train_ts = len(self.data) - TARGET_STEPS
        set_steps = [int(train_ts * spl) for spl in split]  # train + valid
        set_steps += [train_ts - sum(set_steps), TARGET_STEPS]  # test + target

        # create a dict containing the train/valid/test/target sets
        set_name = ["train", "valid", "test", "target"]
        set_data = tf.split(self.data, set_steps, axis=0)
        self.sets = dict(zip(set_name, set_data))

        self.features = dfs[0].columns  # store feature names

    def get(self, subset: str) -> tf.Tensor:
        return self.sets[subset]

In [7]:
dt = DataTensor(df.drop("date"))

# Dataset creation
---

We now turn to building datasets for the subsets of data.

We build individual examples by taking windows of data (a series of successive time-steps) which
we split into two parts, which we call "old" and "new".
The "old" part represents the past values, while the "new" part represents the target (and therefore is always 16 days long).

As we have access to the values of the covariates on the "new" part of the target 
(i.e. we have the oil prices and number of items on promotion for the target time-steps),
we also include these values as part of our input.

In [8]:
FEATURE_COUNT = len(dt.features)

# store the index (along the feature axis) of the target variable ("sales")
TARGET_IDX = dt.features.index("sales")

# store the indices of the covariates
COVS_IND = [dt.features.index(var) for var in dt.features if var != "sales"]

We now define a function that splits a batch of windows into a batch of (inputs, label) pairs.

As explained above, the inputs consists of two part: the "old" values for all variables, 
and the "new" values of the covariates (without the `sales` value, as it is the target).

NOTE: We pass the number of input steps to the function (instead of using -1 in `tf.split()`) so that
the resulting tensors have known shapes.
The `old_vars` tensor therefore has `shape = [None, input_steps, FEATURE_COUNT]` instead
of `[None, None, FEATURE_COUNT]`, which is convenient to initialize models.

In [9]:
# NOTE Need to pass input_length as parameter (instead of using -1 in tf.split)
# NOTE to fully


@tf.function
def split_windows(
    xs: tf.Tensor, input_steps: int
) -> tuple[tuple[tf.Tensor, tf.Tensor], tf.Tensor]:
    # split the windows along the time axis
    old_vars, new_vars = tf.split(xs, [input_steps, TARGET_STEPS], axis=1)

    new_covs = tf.gather(new_vars, indices=COVS_IND, axis=-1)
    new_target = tf.gather(new_vars, indices=[TARGET_IDX], axis=-1)

    return (old_vars, new_covs), new_target

We can now build datasets by windowing the data along the time axis (creating the windows described above)
and then splitting the resulting tensors along the key axis.
We obtain datasets where each example is a window (split by the `split_windows` function) and each dataset
contains the examples for all keys in the time-range specified by the chosen subset.

NOTE: We need to manually set the datasets cardinality by using `repeat().take(...)`, as TensorFlow cannot
know how many elements are in each dataset otherwise due to the windowing.
Not having a known cardinality results in many warnings while training.

In [10]:
class WindowDatasets:
    def __init__(
        self,
        data: DataTensor,
        input_steps: int,
        batch_size: int = 32,
    ):
        self.data = data
        self.input_steps = input_steps
        self.window_steps = input_steps + TARGET_STEPS
        self.batch_size = batch_size

    def make(self, subset: str) -> tf.data.Dataset:
        # card = time-steps, spec = [keys, features]
        ds = tf.data.Dataset.from_tensor_slices(self.data.get(subset))

        # card = windows, spec = [window_steps, keys, features]
        ds = ds.window(size=self.window_steps, shift=1, drop_remainder=True)
        ds = ds.flat_map(lambda window: window.batch(self.window_steps))

        # card = windows, spec = [keys, window_steps, features]
        ds = ds.map(lambda xs: tf.transpose(xs, perm=[1, 0, 2]))

        # card = windows * keys, spec = [window_steps, features]
        ds = ds.flat_map(tf.data.Dataset.from_tensor_slices)

        ds = ds.shuffle(1000).batch(self.batch_size)
        ds = ds.map(
            lambda xs: split_windows(xs, self.input_steps),
            num_parallel_calls=tf.data.AUTOTUNE,
        )
        ds = ds.repeat().take(self.length(subset))  # set the cardinality

        return ds.prefetch(tf.data.AUTOTUNE)

    def length(self, subset: str) -> int:
        shape = self.data.get(subset).shape
        windows_per_key = shape[0] - self.window_steps + 1
        example_count = shape[1] * windows_per_key

        return int(np.ceil(example_count / self.batch_size))

In [11]:
wds = WindowDatasets(dt, input_steps=INPUT_STEPS)

# print element_spec to check that it is as expected
wds.make("train").element_spec

((TensorSpec(shape=(None, 64, 4), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 16, 3), dtype=tf.float32, name=None)),
 TensorSpec(shape=(None, 16, 1), dtype=tf.float32, name=None))

# Model creation
---

## Baseline

For the baseline, we use the most recent past sales values, shifted to have the weekdays match.
This means that we predict the value on a Monday with a value of a Monday, etc...
We do this because we have seen that one week is the most significant frequency in our data, so it 
makes sense that simply taking the most recent 16 time-steps as prediction would be worse that leveraging
this periodicity.

In [12]:
class Baseline(keras.Model):
    def __init__(self):
        super().__init__(name="Baseline")

    def call(self, inputs) -> tf.Tensor:
        old_vars = inputs[0]  # axes = (batch, time, feature)

        old_target = tf.gather(old_vars, [TARGET_IDX], axis=-1)

        # select most recent values with a 5-days shift
        _, pred, _ = tf.split(old_target, [-1, TARGET_STEPS, 5], axis=1)

        return pred

## Decoupling-Coupling model architecture

For the rest of our models, we use an architecture composed of three parts to predict the target values.
The idea is that there should be an underlying variable (call it $X$) so that the sales values $S$
is given by 
$$ S = f(X, Y), $$
where $Y = (Y_1, ..., Y_n)$ are the covariates (oil prices, number of items on promotion, etc.) and $f$ is
some unknown function.

Under this assumption, we proceed in three steps to obtain predictions:
1. decouple the variable $X$ on the "old" time-steps: $X^{old} = g(S^{old}, Y^{old})$,
2. predict the "new" values of the variable $X$ from the "old" values: $\hat{X}^{new} = F(X^{old})$ for some $F$, 
3. couple this "new" value with the covariate to obtain the "new" sales values: $\hat{S}^{new} = f(\hat{X}^{new}, Y^{new})$.

Operating under the assumption that the relationship between $X$, $S$, and the $Y_j$ is not very complicated, 
we use two relatively small dense models to do the decoupling/coupling, and vary the architecure
of the model used for the temporal prediction (step 2).

We define the decoupling/coupling models now.
We initialize the output layer of the coupling model (and therefore the output layer of the entire model)
with ones to make sure that the predictions are not too close to zero, which can happen with random initialization
and prevents learning.

In [13]:
DECOUPNET = keras.Sequential(
    [
        keras.Input(shape=[None, FEATURE_COUNT]),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(1),
    ],
    name="DeCoupNet",
)

COUPNET = keras.Sequential(
    [
        keras.Input(shape=[TARGET_STEPS, FEATURE_COUNT]),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(1, kernel_initializer="ones"),
    ],
    name="CoupNet",
)

Next, we use the functional API to define our model(s), passing as parameter the network used for
the temporal predictions.

In [14]:
def make_model(timenet: keras.Model, name: str) -> keras.Model:
    decoupnet = keras.models.clone_model(DECOUPNET)
    coupnet = keras.models.clone_model(COUPNET)
    concat = keras.layers.Concatenate(axis=-1)

    old_vars = keras.Input(shape=[None, FEATURE_COUNT], name="OldVars")
    new_covs = keras.Input(shape=[TARGET_STEPS, FEATURE_COUNT - 1], name="NewCovs")

    old_Xvar = decoupnet(old_vars)
    new_Xvar = timenet(old_Xvar)
    new_vars = concat([new_Xvar, new_covs])
    new_pred = coupnet(new_vars)

    return keras.Model(inputs=(old_vars, new_covs), outputs=new_pred, name=name)


## TimeNet architectures

We build two models, using a dense and LSTM architecture for the temporal predictions respectively.

In [15]:
timenet_dense = keras.Sequential(
    [
        keras.Input(shape=[INPUT_STEPS, 1]),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(TARGET_STEPS),
        keras.layers.Reshape([TARGET_STEPS, 1]),
    ]
)

dense_model = make_model(timenet_dense, name="Dense")

dense_model.count_params()

17986

In [16]:
timenet_lstm = keras.Sequential(
    [
        keras.Input(shape=[INPUT_STEPS, 1]),
        keras.layers.LSTM(32, return_sequences=True),
        keras.layers.LSTM(32, return_sequences=True),
        keras.layers.LSTM(32, return_sequences=True),
        keras.layers.LSTM(32, return_sequences=True),
        keras.layers.LSTM(TARGET_STEPS, return_sequences=False),
        keras.layers.Reshape([TARGET_STEPS, 1]),
    ]
)

lstm_model = make_model(timenet_lstm, name="LSTM")

lstm_model.count_params()

33186

## Load pretrained models and evaluate

The two models (dense and LSTM) have been trained previously, so we load the weights and evaluate all our models
on the test set.

NOTE: We only load the weights (not the entire models) to avoid the warnings due to missing optimizer variables.
The architectures in the `.keras` files are the ones created above, so this is not an issue.

In [17]:
baseline = Baseline()
baseline.compile(loss="msle")
baseline.evaluate(wds.make("test"))

5068/5068 ━━━━━━━━━━━━━━━━━━━━ 3s 515us/step - loss: 0.3883


0.3344363570213318

In [18]:
dense_model.load_weights(os.path.join("models", "sk_dense.keras"))
dense_model.compile(loss="msle")
dense_model.evaluate(wds.make("test"))

5068/5068 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.2071


0.19476844370365143

In [19]:
lstm_model.load_weights(os.path.join("models", "sk_lstm.keras"))
lstm_model.compile(loss="msle")
lstm_model.evaluate(wds.make("test"))

5068/5068 ━━━━━━━━━━━━━━━━━━━━ 76s 15ms/step - loss: 0.4197


0.3992605209350586

The dense model presents a good improvement on the baseline, while the LSTM model seems to be the worst of the three.
However, as this evaluation is computed on average over all keys, it is heavily skewed by the keys with the highest loss
and therefore does not reflect the value of each model, which can all outperform the others on a specific set of keys.

# Evaluation on most recent window
---

We now evaluate our models on the last window of the training set, that is by predicting the last 16 days for which we 
know the `sales` values.

Evaluating on the most recent data gives a good idea of how each model will perform on the target time-steps, which are right
after.
Moreove, we evaluate the models on a per-key basis to identify where each model shines, and which keys are the most difficult 
to predict.
This is where we see the shortcomings of our initial choice of not including the key data in the input, as some
keys (or more generally some families) follow specific patterns (e.g. time periodicity), which our models have no
way of predicting since they do not know which key they are predicting.


We define an `evaluate` function which returns the loss per key of a model on the last training window.

In [20]:
def evaluate(model: keras.Model, input_steps: int = INPUT_STEPS) -> np.ndarray:
    window_steps = input_steps + TARGET_STEPS

    # fetch last window before target
    _, window, _ = tf.split(dt.data, [-1, window_steps, TARGET_STEPS], axis=0)

    # use key axis as batch -> (key, time, feature)
    window = tf.transpose(window, [1, 0, 2])

    inputs, target = split_windows(window, input_steps=input_steps)
    preds = model(inputs)

    # remove feature axis -> [TARGET_STEPS, keys]
    target, preds = tf.squeeze(target), tf.squeeze(preds)

    return MSLE(target, preds).numpy()

We now evaluate each of our models.
We take advantage of the fact that the LSTM model accepts inputs of various length to use a longer
input here, which slightly improves performance.

We store the results in the `loss_df` dataframe, which contains one row per key and one column per model.

In [21]:
# NOTE Take advantage of the fact that the LSTM model accepts inputs of different lengths
# NOTE to use longer inputs. This results in slightly better performance.

loss_df = pl.DataFrame(
    {
        "Baseline": evaluate(baseline),
        "Dense": evaluate(dense_model),
        "LSTM": evaluate(lstm_model, input_steps=128),
    }
)

loss_df = pl.concat(
    [df.select("store_nbr", "family").unique(maintain_order=True), loss_df],
    how="horizontal",
)

We now plot the spread of the loss per family for each model.

We can see that certain families (most notably "SCHOOL AND OFFICE SUPPLIES", which is highly yearly periodic)
are difficult to predict for all models, and that some models (mostly Dense) perform better than others on certain families.

Overall, there are quite large perfromance discrepencies between families, which again can be explained by the vast
differences in the relevant time-frequencies from one family to the next.

In [22]:
px.box(loss_df, y=["Baseline", "Dense", "LSTM"], color="family").update_layout(
    showlegend=False,
    height=400,
    title="Model loss spread per family",
).update_xaxes(title=None).update_yaxes(type="log", title="log-loss")

Similarly, we plot the loss but grouped by store.

Here, we observe much more uniform results, which confirms that there are no significant differences from
one store to the next and that the high variance in our predictions stems from the variablity in families.

In [23]:
px.box(loss_df, y=["Baseline", "Dense", "LSTM"], color="store_nbr").update_layout(
    showlegend=False,
    height=400,
    title="Model loss spread per store_nbr",
).update_xaxes(title=None).update_yaxes(type="log", title="log-loss")

To get a quantified measure of performance, we look at the statistics per model.
We include a column `min_loss` containing the smallest loss (across all models) as a measure of what we can
achieve if we select the best model to predict each key.

In [24]:
# compute the min loss per key
loss_df = loss_df.with_columns(
    pl.min_horizontal("Baseline", "Dense", "LSTM").alias("min_loss")
)

# print statistics
loss_df.drop("store_nbr", "family").describe()[2:]

statistic,Baseline,Dense,LSTM,min_loss
str,f64,f64,f64,f64
"""mean""",0.40,0.21,0.41,0.17
"""std""",1.47,0.39,0.67,0.38
"""min""",0.00,0.01,0.01,0.00
"""25%""",0.05,0.05,0.14,0.04
"""50%""",0.16,0.15,0.26,0.10
"""75%""",0.42,0.27,0.49,0.22
"""max""",24.95,8.08,9.22,8.08


# Composite model
---

We now determine the best model for each key and use this to create a "composite model".

We begin by determining how many keys should be predicted by each model.
Unsurprisingly in view of the model statistics, the Dense model is the best model for the most keys by far,
but we also observe that the LSTM model actually outperforms the baseline in terms of how number of keys.

It is also worth noting that most of the keys on which the baseline performs best are identically zero.

In [25]:
{
    name: loss_df.select(pl.col(name) == pl.col("min_loss")).sum().item()
    for name in ["Baseline", "Dense", "LSTM"]
}

{'Baseline': 326, 'Dense': 1121, 'LSTM': 335}

We plot the losses per family for the composite model and observe that for most families, all losses are below 1.0.
This represents an improvement compared to the plot per family above, where each model had several families go above 1.0.

In [26]:
px.box(loss_df, y="min_loss", color="family").update_layout(
    height=400,
    title="Composite model loss spread per family",
).update_xaxes(title=None).update_yaxes(type="log", title="log-loss")

We now look into the worst keys for the composite model.
These are keys for which none of our models have managed a good prediction.

In [27]:
def plot_worst_key(count: int, input_steps: int = INPUT_STEPS):
    wks = loss_df.sort(by="min_loss")[-count:]
    fig = go.Figure()

    for row in range(count):
        store_nbr, family = wks[row]["store_nbr"].item(), wks[row]["family"].item()
        kdf = (
            df.filter((pl.col("store_nbr") == store_nbr) & (pl.col("family") == family))
            .head(-TARGET_STEPS)
            .tail(input_steps + TARGET_STEPS)
        )
        fig.add_scatter(x=kdf["date"], y=kdf["sales"], name=f"{store_nbr}, {family}")

    # need the x value as timestamp in ms because of a plotly bug
    # see https://github.com/plotly/plotly.py/issues/3065
    x_vline = (
        df["date"]
        .unique(maintain_order=True)
        .head(-TARGET_STEPS)
        .tail(input_steps + TARGET_STEPS)
        .dt.timestamp("ms")[input_steps]
    )

    fig.add_vline(
        x=x_vline,
        line_dash="dash",
        annotation_text="target start",
        annotation_position="bottom",
        annotation_textangle=-45,
    )

    fig.update_layout(title=f"Sales of worst {count} keys on the last window")
    fig.update_yaxes(showticklabels=False)
    fig.show()

Plotting the `sales` value on the last training window for the 20 worst keys, we observe that most of these
exhibit significant differences between the "old" time-steps (before the start of the target) and the "new"
time-steps (the target).
In fact, most of these are from the "SCHOOL AND OFFICE SUPPLIES" family, which has a peak once a year, around the 
start of the school year (and this is the worst-performing family by far for our models, as can be seen in the boxplots above).

Because of the great difference between the values used as input and the target values, our models have no hope of predicting
such spikes.
To remedy this issue, we would need our models to learn that certain keys (or families) possess this kind of periodicity, 
which does require passing information about the key to the models as input.

In [28]:
plot_worst_key(20)

# Predictions and submission
---

We now prepare the submission file containing the predictions for the target time-steps.

We read the target dataframe `test.csv` and add a `sales` column with our predictions.
This allows us to have the keys in the correct order (recall that the `store_nbr` column is ordered weird)
and gives us the values of the `id` column, which are needed for the submission.

In [29]:
target_df = pl.read_csv(os.path.join("input", "test.csv"), try_parse_dates=True)

In [30]:
TARGET_DATES = target_df["date"].unique(maintain_order=True).cast(pl.String)
KEYS = df.select("store_nbr", "family").unique(maintain_order=True)


def predict(model: keras.Model, input_steps: int = INPUT_STEPS) -> pl.DataFrame:
    # fetch last window of the data (including target time-steps)
    _, window = tf.split(dt.data, [-1, input_steps + TARGET_STEPS], axis=0)

    # use key axis as batch axis -> (key, time, feature)
    window = tf.transpose(window, perm=[1, 0, 2])
    inputs, _ = split_windows(window, input_steps)

    # make dataframe with key as rows and time-steps as cols
    preds = pl.DataFrame(tf.squeeze(model(inputs)).numpy())

    # set the column names to the date and add "store_nbr" and "family" columns
    preds = preds.rename({col.name: TARGET_DATES[idx] for idx, col in enumerate(preds)})
    preds = pl.concat([KEYS, preds], how="horizontal")

    return preds

In [31]:
# compute the prediction for all keys for all models
model_preds = {
    "Baseline": predict(baseline),
    "Dense": predict(dense_model),
    "LSTM": predict(lstm_model, input_steps=128),
}

For each model, we select the predictions on the keys on which the model is the best among all our models.
We concatenate the results to obtain the predictions of the composite model.

In [32]:
preds = []

for model_name, model_pred in model_preds.items():
    keys_pred = (
        loss_df.filter(pl.col(model_name) == pl.col("min_loss"))
        .select("store_nbr", "family")
        .join(model_pred, on=["store_nbr", "family"])
    )
    preds.append(keys_pred)

# concatenate the predictions for each model and unpivot
predictions = (
    pl.concat(preds)
    .unpivot(index=["store_nbr", "family"], variable_name="date", value_name="sales")
    .cast({"date": pl.Date})
)

## Prepare and save submissions

In [33]:
predictions = target_df.join(predictions, on=["date", "store_nbr", "family"])

In [34]:
submission = predictions.select("id", "sales")
submission.write_csv("submission.csv")

This submission obtains a score (RMSLE) of 0.42634.

This correspond to a loss (MSLE) of about 0.18, which is consistent with what our compound model attained on the last 16 days of the training set (MSLE of 0.17).